In [ ]:
import pandas as pd
import re
import string

df=pd.read_json("./data/arxiv-metadata-oai-snapshot.json",lines=True) #download from https://www.kaggle.com/datasets/Cornell-University/arxiv

def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

df=df[["abstract","update_date"]]
df['year'] = pd.to_datetime(df['update_date'], infer_datetime_format=True)
df['year'] = df['year'].dt.to_period('Y')
df=df.sort_values(by="year")


df=df[["abstract","year"]]
df['abstract'] = df['abstract'].str.replace(r'@\w+', '')
df['abstract'] = df['abstract'].str.replace('\n', ' ').replace('\r', '')
df['abstract'] = df['abstract'].apply(lambda x: remove_punct(x))
df=df.dropna()
df=df.sort_values(by="year")
df=df.reset_index(drop=True)
df=df.reset_index()

In [ ]:
from models.antm import ANTM
window_size=3
overlap=1

#take a random sample for example
dt=df.sample(n = 1000)
dt=dt.sort_values("year")
dt=dt.reset_index(drop=True)
dt=dt.reset_index()

evolving_topics=ANTM(dt,overlap,window_size,mode="data2vec",num_words=10)